In [1]:
# file_properties_df

In [20]:
import csv
from datetime import datetime,timedelta
from nna import fileUtils
def read_csv(csv_file_path, fieldnames=None):
    with open(csv_file_path, encoding='utf-8') as csv_file:
        rows = csv.DictReader(csv_file,fieldnames=fieldnames)
        rows = list(rows)
        for row in rows:
            for key,value in row.items():
                if value.lower() == 'false':
                    row[key]='FALSE'

    return rows

def write_csv(new_csv_file, rows_new,fieldnames=None):
    with open(new_csv_file, 'w', newline='',encoding='utf-8') as csvfile:
        if fieldnames is None:
            fieldnames = rows_new[0].keys()
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        writer.writerows(rows_new)
        


def get_region(row):
    site_id = int(row['Site ID'])
    if site_id<31:
        region='prudhoe'
    else:
        region='anwr'
    return region


In [21]:
csv_file='/scratch/enis/data/nna/labeling/megan/datasetV3.1.csv'
rows=read_csv(csv_file)


In [22]:
# c=0
# t=0
import pandas as pd
from pathlib import Path

file_properties_df = pd.read_pickle(
    '/scratch/enis/data/nna/database/allFields_dataV6.pkl')
output_folder = '/scratch/enis/data/nna/labeling/megan/datasetV2SpltMono/dataset_exact/'


In [24]:
# file_properties_df

In [25]:
old_path2new_path={}
# row

In [58]:
len(old_path2new_path)

1671

In [26]:
c=0
failed=[]
for row in rows:
    region=get_region(row)
    location = row['Site ID']
    file_name='_'.join(([region,location,Path(row['File Name']).stem]))
    
    style='%H:%M:%S.%f'
    length_time = datetime.strptime(row['Length'], style)
    length_time=(length_time - length_time.replace(
            hour=0, minute=0, second=0, microsecond=0)).total_seconds()
    if length_time<10:
        start = ' '.join([row['Date'],row['Start Time']])
        start_time = datetime.strptime(start, '%m/%d/%Y %H:%M:%S.%f')
        end_time = start_time+timedelta(seconds=10)
        length=10
        buffer=0
        filenames,sorted_filtered=fileUtils.query_audio(location,
                region,
                start_time,
                end_time,
                length,
                buffer,
                file_properties_df,
                file_name,
                display_flag=False,
                save=True,
                tmp_folder=output_folder,
                version='V2')
        if c%100==0:
            print(c)
        c+=1
        if filenames is None:
            print('ERROR')
            print(sorted_filtered)
            print(row)
        if filenames:
            old_path2new_path[row['Clip Path']]=filenames[0]
        else:
            failed.append(row)


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600


In [27]:
c=0
# failed=[]
failed_again=[]
old_path2new_path_2={}
for row in failed:
    region=get_region(row)
    location = row['Site ID']
    file_name='_'.join(([region,location,Path(row['File Name']).stem]))
    
    style='%H:%M:%S.%f'
    length_time = datetime.strptime(row['Length'], style)
    length_time=(length_time - length_time.replace(
            hour=0, minute=0, second=0, microsecond=0)).total_seconds()
    if length_time<10:
        print(length_time)
        start = ' '.join([row['Date'],row['Start Time']])
        start_time = datetime.strptime(start, '%m/%d/%Y %H:%M:%S.%f')
        end_time = start_time+timedelta(seconds=length_time)
        start_time = end_time-timedelta(seconds=10)
        length=10
        buffer=0
        filenames,sorted_filtered=fileUtils.query_audio(location,
                region,
                start_time,
                end_time,
                length,
                buffer,
                file_properties_df,
                file_name,
                display_flag=False,
                save=True,
                tmp_folder=output_folder,
                version='V2',
                stereo2mono=True,
                sampling_rate=48000)
        if c%100==0:
            print(c)
        c+=1
        if filenames is None:
            print('ERROR')
            print(sorted_filtered)
            print(row)
        if filenames:
            old_path2new_path_2[row['Clip Path']]=filenames[0]
        else:
            failed_again.append(row)


9.0
0
7.995354
9.284396
8.099375
7.302354
5.230583
3.951104
1.030146
2.764458
4.016292
1.356146
5.167
6.337396
9.206167
5.998354
5.789708
2.060313
1.22575
6.885083
5.450688
6.92075
6.862188
5.062667
5.114854
5.766854
9.388729
5.021958
2.030563


In [55]:
# failed_again[20]

In [59]:
old_path2new_path_c = old_path2new_path.copy()

In [17]:
len(failed)

28

In [18]:
len(old_path2new_path)

1671

In [28]:
rows_dict={row['Clip Path']:row for row in rows}


In [29]:
# old_path2new_path

for key,new_key in old_path2new_path_2.items():
    rows_dict[key]['Clip Path']=new_key
    rows_dict[key]['Comments']+=',_exact_'

In [30]:
for key,new_key in old_path2new_path.items():
    rows_dict[key]['Clip Path']=new_key
    rows_dict[key]['Comments']+=',_exact_'

In [31]:
rows_exact = list(rows_dict.values())

In [32]:
write_csv('/scratch/enis/data/nna/labeling/megan/datasetV3.1.exact.csv',rows_exact)
# rows_exact = read_csv('/scratch/enis/data/nna/labeling/megan/datasetV3.1.exact.csv')

In [33]:
# !head '/scratch/enis/data/nna/labeling/megan/datasetV3.1.exact.csv'


In [35]:
# !head '/scratch/enis/data/nna/labeling/megan/datasetV3.1.csv'
# import importlib
# importlib.reload(nna.fileUtils)
# importlib.reload(nna.labeling_utils)

from nna import fileUtils

In [36]:
# for row in rows_exact:
#     if 'flac' in row['Clip Path'] and 'exact' in row['Clip Path']:
#         print(row['Clip Path'])


In [56]:
ff='/scratch/enis/data/nna/labeling/megan/datasetV2SpltMono/dataset_exact/anwr_48_S4A10252_20190709_213000_exact_2019-07-09_22-40-15_0m_0s__0m_10s.flac'
for key,value in old_path2new_path.items():
    if value==ff:
        print(key)


/scratch/enis/data/nna/labeling/megan/datasetV2SpltMono/AudioSamplesPerSite/Site_48/S4A10252_20190709_213000_Bio_Bird_DGS03_splt-0.wav
/scratch/enis/data/nna/labeling/megan/datasetV2SpltMono/AudioSamplesPerSite/Site_48/S4A10252_20190709_213000_Bio_Bird_SongB_splt-0.wav


In [61]:
list2=list(old_path2new_path.values())
# old_path2new_path_filter={}
# list2=[row['Clip Path'] for row in rows_exact]
to_delete=[]
len(list2),len(set(list2))
from collections import Counter
for item in Counter(list2).items():
    if (item[1]>1):
        print(item)
        for k,v in old_path2new_path.items():
            if v==item[0]:
                to_delete.append(k)

('/scratch/enis/data/nna/labeling/megan/datasetV2SpltMono/dataset_exact/anwr_48_S4A10252_20190709_213000_exact_2019-07-09_22-40-15_0m_0s__0m_10s.flac', 2)
('/scratch/enis/data/nna/labeling/megan/datasetV2SpltMono/dataset_exact/prudhoe_30_S4A10264_20190608_041602_exact_2019-06-08_04-19-02_0m_0s__0m_10s.flac', 2)
('/scratch/enis/data/nna/labeling/megan/datasetV2SpltMono/dataset_exact/prudhoe_30_S4A10264_20190608_041602_exact_2019-06-08_04-25-06_0m_0s__0m_10s.flac', 2)
('/scratch/enis/data/nna/labeling/megan/datasetV2SpltMono/dataset_exact/prudhoe_12_S4A10268_20190606_091602_exact_2019-06-06_10-20-13_0m_0s__0m_10s.flac', 2)
('/scratch/enis/data/nna/labeling/megan/datasetV2SpltMono/dataset_exact/prudhoe_22_S4A10275_20190806_043000_exact_2019-08-06_05-18-03_0m_0s__0m_10s.flac', 2)
('/scratch/enis/data/nna/labeling/megan/datasetV2SpltMono/dataset_exact/prudhoe_22_S4A10275_20190806_043000_exact_2019-08-06_05-20-41_0m_0s__0m_10s.flac', 2)
('/scratch/enis/data/nna/labeling/megan/datasetV2SpltMo

In [63]:
for key in to_delete:
    del old_path2new_path[key]

In [98]:
def swap_file_w_monosound_set_sr(input_file,
                                 sr=48000,
                                 suffix='.wav',
                                 backend_path='/scratch/enis/conda/envs/speechEnv/bin/ffmpeg',):

    input_file_new_suffix=input_file.with_suffix(suffix)
    if not input_file.exists():
        print('File does not exists',input_file)
    output_file= Path('./temp.wav').with_suffix(suffix)
    fileUtils.make_mono_set_sr(
            str(input_file),
            str(output_file),
            stereo2mono=True,
            sampling_rate=sr,
            backend_path=backend_path,
            overwrite=True)
    input_file.unlink()
    shutil.move(output_file, input_file_new_suffix)
    return input_file_new_suffix

In [66]:
from pathlib import Path
import shutil
c=0
for row in rows_exact:
    if 'exact' not in row['Clip Path']:
        continue
    input_file=Path(row['Clip Path'])
    input_file_wav=input_file.with_suffix('.wav')
    if not input_file.exists():
        continue
    output_file=('./temp.wav')
    fileUtils.make_mono_set_sr(
            str(input_file),
            str(output_file),
            stereo2mono=True,
            sampling_rate=48000,
            backend_path='/scratch/enis/conda/envs/speechEnv/bin/ffmpeg',
            overwrite=True)
    input_file_wav=input_file.with_suffix('.wav')
    input_file.unlink()
    shutil.move(output_file, input_file_wav)
    c+=1
    if c%100==0:
        print(c)
    


100
200
300
400
500
600
700
800
900
1000
1100


In [69]:
Path('asd.mp3').suffix

'.mp3'

In [ ]:
for row in rows_exact:
    if 'exact' in row['Clip Path']:
        input_file=Path(row['Clip Path'])
        if input_file.suffix=='.flac':
            # print(input_file)
        # continue
            input_file_wav=input_file.with_suffix('.wav')
            if not input_file_wav.exists():
                raise Error('asd')
            row['Clip Path'] = input_file_wav
            

In [68]:
# !/scratch/enis/conda/envs/speechEnv/bin/python /home/enis/projects/nna/src/nna/slack_message.py -t "cpu job ended" -m 'split audio ended'


In [72]:
write_csv('/scratch/enis/data/nna/labeling/megan/datasetV3.1.exact2.csv',rows_exact)

In [74]:
# !head /scratch/enis/data/nna/labeling/megan/datasetV3.1.exact2.csv

In [99]:
import json
from subprocess import check_output

def get_media_info(filename, print_result=False):
    """
    Returns:
        result = dict with audio info where:
        result['format'] contains dict of tags, bit rate etc.
        result['streams'] contains a dict per stream with sample rate, channels etc.
    """
    filename = str(filename)
    result = check_output(['/scratch/enis/conda/envs/speechEnv/bin/ffprobe',
                            '-hide_banner', '-loglevel', 'panic',
                            '-show_format',
                            '-show_streams',
                            '-of',
                            'json', filename])

    result = json.loads(result)

    if print_result:
        print('\nFormat')

        for key, value in result['format'].items():
            print('   ', key, ':', value)

        print('\nStreams')
        for stream in result['streams']:
            for key, value in stream.items():
                print('   ', key, ':', value)

        print('\n')

    return result

'10.000000'

In [92]:
ff='/scratch/enis/data/nna/labeling/megan/datasetV2SpltMono/dataset_exact/anwr_48_S4A10252_20190709_213000_exact_2019-07-09_22-40-15_0m_0s__0m_10s.wav'
a=get_media_info(ff)

for row in rows_exact:
    input_file=Path(row['Clip Path'])
    a=get_media_info(input_file)
    if len(a['streams'])>1:
        print(input_file)
    if a['streams'][0]['sample_rate']!='48000':
        print(a['streams'][0]['sample_rate'],input_file,)
    if a['streams'][0]['duration']!='10.000000':
        print(a['streams'][0]['duration'],input_file,'duration')
# a


5.000000 /scratch/enis/data/nna/labeling/megan/datasetV2SpltMono/dataset_exact/prudhoe_29__exact_2019-07-29_00-15-55_0m_0s__0m_10s.wav duration


In [79]:
# /scratch/enis/data/nna/labeling/megan/datasetV2SpltMono/dataset_exact/prudhoe_29__exact_2019-07-29_00-15-55_0m_0s__0m_10s.wav
# 5

'/scratch/enis/data/nna/labeling/megan/datasetV2SpltMono/dataset_exact/anwr_48_S4A10252_20190709_213000_exact_2019-07-09_22-40-15_0m_0s__0m_10s.flac'

In [81]:
!ls '/scratch/enis/data/nna/labeling/megan/datasetV2SpltMono/dataset_exact/anwr_48_S4A10252_20190709_213000_exact_2019-07-09_22-40-15_0m_0s__0m_10s.wav'

/scratch/enis/data/nna/labeling/megan/datasetV2SpltMono/dataset_exact/anwr_48_S4A10252_20190709_213000_exact_2019-07-09_22-40-15_0m_0s__0m_10s.wav


In [93]:
a_dataset=read_csv('/scratch/enis/data/nna/labeling/megan/datasetV4.csv')

In [96]:
# len(a_dataset),a_dataset[0]

In [ ]:
c=0
short_files=[]
for row in a_dataset:
    input_file=row['Clip Path']
    a=get_media_info(input_file)
    needs_change=False
    if len(a['streams'])>1:
        # print('streams',len(a['streams']),input_file)
        needs_change=True
    if a['streams'][0]['sample_rate']!='48000':
        # print(a['streams'][0]['sample_rate'],input_file,)
        needs_change=True
    if a['streams'][0]['duration']!='10.000000':
        print(a['streams'][0]['duration'],input_file,'duration')
        short_files.append(input_file)
        # needs_change=True
    if needs_change:
        swap_file_w_monosound_set_sr(input_file)
    if c%100==0:
        print(c)
    c+=1


0
100
200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
9.999833 /scratch/enis/data/nna/labeling/megan/yfitloiq-V1/dalton/04/S4A10281_20190422_212934_3m_0s__3m_10s.wav duration
9.999833 /scratch/enis/data/nna/labeling/megan/yfitloiq-V1/dalton/05/S4A10454_20191014_064000_2m_40s__2m_50s.wav duration
9.999833 /scratch/enis/data/nna/labeling/megan/yfitloiq-V1/dalton/04/S4A10281_20190401_224934_4m_20s__4m_30s.wav duration
6900
7000
7100
7200
9.999833 /scratch/enis/data/nna/labeling/megan/yfitloiq-V1/dalton/04/S4A10281_20190328_184934_4m_0s__4m_10s.wav duration
7300
9.999833 /scratch/enis/data/nna/labeling/megan/yfitloiq-V1/dalton/05/S4A10454_20191022_104000_4m_20s__4m_30s.wav duration
10.000167 /scratch/enis/data/nna/labeling/megan/yfitloiq-V1/dalton/05/S4A1045

In [101]:
len(short_files)

31